In [3]:
!pip install roboflow
from roboflow import Roboflow
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os
from PIL import Image, ImageDraw
import pandas as pd


import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import ADASYN
import pickle
pd.set_option('display.max_columns', None)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
rf = Roboflow(api_key="xw35YyNnvd19WHlauQev")
project = rf.workspace().project("gp-sinuses")
model_box = project.version(7).model
project = rf.workspace().project("gp-segmentation")
model_seg = project.version(4).model

loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...


#functions

In [6]:
       
def p_ratio(image):       
      # gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      #range of gray values
      value_min = 28
      value_max = 94
      mask = np.logical_and(image >= value_min, image <= value_max)
      count_gray = np.count_nonzero(mask)

      #range of black values
      value_min = 0
      value_max = 20
      mask = np.logical_and(image >= value_min, image <= value_max)
      count_black = np.count_nonzero(mask)
      
      if (count_black==0 and count_gray==0):
          ratio=0.5
          return ratio

      ratio=count_gray/(count_black+count_gray)
      return ratio

      # print(ratio)

In [7]:
def area(image):
      height, width = image.shape
      area=height*width
      return area

In [8]:
def count_gray(image):
      value_min = 28
      value_max = 94
      mask = np.logical_and(image >= value_min, image <= value_max)
      count_gray = np.count_nonzero(mask)
      return count_gray


In [9]:
def count_black(image):
      value_min = 0
      value_max = 20
      mask = np.logical_and(image >= value_min, image <= value_max)
      count_black = np.count_nonzero(mask)
      return count_black

In [10]:
def class_ratio(image,Class):       
      if(Class=="maxillary R"):
        count_maxillary_R.append(p_ratio(image))

      elif(Class=="maxillary L"):
        count_maxillary_L.append(p_ratio(image))

      elif(Class=="frontal"):
        count_frontal.append(p_ratio(image))
          
      elif(Class=="ant.ethmoid R"):
        count_int_R.append(p_ratio(image))

      elif(Class=="ant.ethmoid L"):
        count_int_L.append(p_ratio(image))

      elif(Class=="post. ethmoid R"):
        count_post_R.append(p_ratio(image))

      elif(Class=="post. ethmoid L"):
        count_post_L.append(p_ratio(image))

      elif(Class=="sphenoid R"):
        count_sphenoid_R.append(p_ratio(image))

      elif(Class=="sphenoid L"):
        count_sphenoid_L.append(p_ratio(image))

      elif(Class=="septum"):
        count_septum.append(area(image))

      elif(Class=="inferior turbinate"):
        count_inferior_turbinate.append(p_ratio(image))

      elif(Class=="concha bullosa"):
        count_concha_black.append(count_black(image))
        count_concha_size.append(count_gray(image)+count_black(image))

      elif(Class=="nasopharynx"):
          count_nasopharynx.append(count_black(image))

In [11]:
def box(image):
  results = model_box.predict(image).json()
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  for index, prediction in enumerate(results["predictions"]):

    roi_x = int(prediction['x'] - prediction['width'] / 2)
    roi_y = int(prediction['y'] - prediction['height'] / 2)
    roi_width = int(prediction['width'])
    roi_height = int(prediction['height'])

    roi = gray_image[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width]
          

    class_ratio(roi,prediction["class"])


In [12]:
def seg(file_path):
  results = model_seg.predict(file_path).json()
  image = cv2.imread(file_path)

  for index, prediction in enumerate(results["predictions"]):

    polygon = [(point_obj["x"], point_obj["y"]) for point_obj in prediction["points"]]

    maskIm = Image.new('L', (image.shape[1], image.shape[0]), 255)
    ImageDraw.Draw(maskIm).polygon(polygon, outline=1, fill=1)

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    new_img=gray_image*maskIm

    roi_x = int(prediction['x'] - prediction['width'] / 2)
    roi_y = int(prediction['y'] - prediction['height'] / 2)
    roi_width = int(prediction['width'])
    roi_height = int(prediction['height'])

    roi = new_img[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width]

    # cv2_imshow(new_img)
    # cv2_imshow(roi)

    class_ratio(roi,prediction["class"])

#process full case

In [ ]:

col=['maxillaryR score','maxillary R gray/black ratio','m_r_std','m_r_1','m_r_2','m_r_3','m_r_4','m_r_5','maxillary L score','maxillary L gray/black ratio','m_l_std','m_l_1','m_l_2','m_l_3','m_l_4','m_l_5','frontal score','frontal gray/black ratio','front_std','front_1','front_2','front_3','front_4','front_5','ant.ethmoid R score','ant.ethmoid R gray/black ratio','ant_r_std','ant_r_1','ant_r_2','ant_r_3','ant_r_4','ant_r_5','ant.ethmoid L score','ant.ethmoid L gray/black ratio','ant_l_std','ant_l_1','ant_l_2','ant_l_3','ant_l_4','ant_l_5','post.ethmoid R score','post.ethmoid R gray/black ratio','post_r_std','post_r_1','post_r_2','post_r_3','post_r_4','post_r_5','post.ethmoid L score','post.ethmoid L gray/black ratio','post_l_std','post_l_1','post_l_2','post_l_3','post_l_4','post_l_5','sphenoid R score','sphenoid R gray/black ratio','sphen_r_std','sphen_r_1','sphen_r_2','sphen_r_3','sphen_r_4','sphen_r_5','sphenoid L score','sphenoid L gray/black ratio','sphen_l_std','sphen_l_1','sphen_l_2','sphen_l_3','sphen_l_4','sphen_l_5','Osteomeatal complex R score','Osteomeatal complex L score','septum score','septum area','sept_std','sept_1','sept_2','sept_3','sept_4','sept_5',
               'inferior turbinate score','inferior turbinate ratio','it_std','it_1','it_2','it_3','it_4','it_5','concha bullosa score','concha_blk_ratio','concha_blk_std','concha_blk_1','concha_blk_2','concha_blk_3','concha_blk_4','concha_blk_5',
                 'concha_size_ratio','concha_size_std','concha_size_1','concha_size_2','concha_size_3','concha_size_4','concha_size_5','nasopharynx score','nasopharynx area','naso_std','naso_1','naso_2','naso_3','naso_4','naso_5']
data = pd.DataFrame()
for i in col:
  data[i]=np.nan

In [36]:
data

,maxillaryR score,maxillary R gray/black ratio,m_r_std,m_r_1,m_r_2,m_r_3,m_r_4,m_r_5,maxillary L score,maxillary L gray/black ratio,m_l_std,m_l_1,m_l_2,m_l_3,m_l_4,m_l_5,frontal score,frontal gray/black ratio,front_std,front_1,front_2,front_3,front_4,front_5,ant.ethmoid R score,ant.ethmoid R gray/black ratio,ant_r_std,ant_r_1,ant_r_2,ant_r_3,ant_r_4,ant_r_5,ant.ethmoid L score,ant.ethmoid L gray/black ratio,ant_l_std,ant_l_1,ant_l_2,ant_l_3,ant_l_4,ant_l_5,post.ethmoid R score,post.ethmoid R gray/black ratio,post_r_std,post_r_1,post_r_2,post_r_3,post_r_4,post_r_5,post.ethmoid L score,post.ethmoid L gray/black ratio,post_l_std,post_l_1,post_l_2,post_l_3,post_l_4,post_l_5,sphenoid R score,sphenoid R gray/black ratio,sphen_r_std,sphen_r_1,sphen_r_2,sphen_r_3,sphen_r_4,sphen_r_5,sphenoid L score,sphenoid L gray/black ratio,sphen_l_std,sphen_l_1,sphen_l_2,sphen_l_3,sphen_l_4,sphen_l_5,Osteomeatal complex R score,Osteomeatal complex L score,septum score,septum area,sept_std,sept_1,sept_2,sept_3,sept_4,sept_5,inferior turbinate score,inferior turbinate ratio,it_std,it_1,it_2,it_3,it_4,it_5,concha bullosa score,concha_blk_ratio,concha_blk_std,concha_blk_1,concha_blk_2,concha_blk_3,concha_blk_4,concha_blk_5,concha_size_ratio,concha_size_std,concha_size_1,concha_size_2,concha_size_3,concha_size_4,concha_size_5,nasopharynx score,nasopharynx area,naso_std,naso_1,naso_2,naso_3,naso_4,naso_5


In [ ]:
#i is index row you want to fill
id=data['ID']

for n in id:

  count_maxillary_R=[0]*6
  count_maxillary_L=[0]*6
  count_frontal=[0]*6
  count_int_R=[0]*6
  count_int_L=[0]*6
  count_post_R=[0]*6
  count_post_L=[0]*6
  count_sphenoid_R=[0]*6
  count_sphenoid_L=[0]*6
  count_septum=[0]*6
  count_inferior_turbinate=[0]*6
  count_concha_ratio=[0]*6
  count_concha_black=[0]*6
  count_concha_size=[0]*6
  count_nasopharynx=[0]*6

  directory_path = f"/content/drive/MyDrive/blue flash finishedd/{n}"   #folder path


  # Iterate over each file in the directory
  for file_name in os.listdir(directory_path):
        # Check if the file is a picture (you can customize the condition based on the file extensions of your pictures)
        if file_name.endswith(".jpg") or file_name.endswith(".JPG"):
            # Process each picture file here
            file_path = os.path.join(directory_path, file_name)
            

            

            image = cv2.imread(file_path)

            box(image)

            seg(file_path)

            # calculate()

            # fill(n)
            # n=n+1


  data.loc[i:i, 'maxillary R gray/black ratio']=np.mean(count_maxillary_R)
  data.loc[i:i, 'm_r_std']=np.std(count_maxillary_R)
  data.loc[i:i, 'm_r_1']=sorted(count_maxillary_R, reverse=True)[:1]
  data.loc[i:i, 'm_r_2']=sorted(count_maxillary_R, reverse=True)[1:2]
  data.loc[i:i, 'm_r_3']=sorted(count_maxillary_R, reverse=True)[2:3]
  data.loc[i:i, 'm_r_4']=sorted(count_maxillary_R, reverse=True)[3:4]
  data.loc[i:i, 'm_r_5']=sorted(count_maxillary_R, reverse=True)[4:5]

  data.loc[i:i, 'maxillary L gray/black ratio']=np.mean(count_maxillary_L)
  data.loc[i:i, 'm_l_std']=np.std(count_maxillary_L)
  data.loc[i:i, 'm_l_1']=sorted(count_maxillary_L, reverse=True)[:1]
  data.loc[i:i, 'm_l_2']=sorted(count_maxillary_L, reverse=True)[1:2]
  data.loc[i:i, 'm_l_3']=sorted(count_maxillary_L, reverse=True)[2:3]
  data.loc[i:i, 'm_l_4']=sorted(count_maxillary_L, reverse=True)[3:4]
  data.loc[i:i, 'm_l_5']=sorted(count_maxillary_L, reverse=True)[4:5]




  data.loc[i:i, 'ant.ethmoid R gray/black ratio']=np.mean(count_int_R)
  data.loc[i:i, 'ant_r_std']=np.std(count_int_R)
  data.loc[i:i, 'ant_r_1']=sorted(count_int_R, reverse=True)[:1]
  data.loc[i:i, 'ant_r_2']=sorted(count_int_R, reverse=True)[1:2]
  data.loc[i:i, 'ant_r_3']=sorted(count_int_R, reverse=True)[2:3]
  data.loc[i:i, 'ant_r_4']=sorted(count_int_R, reverse=True)[3:4]
  data.loc[i:i, 'ant_r_5']=sorted(count_int_R, reverse=True)[4:5]

  data.loc[i:i, 'ant.ethmoid L gray/black ratio']=np.mean(count_int_L)
  data.loc[i:i, 'ant_l_std']=np.std(count_int_L)
  data.loc[i:i, 'ant_l_1']=sorted(count_int_L, reverse=True)[:1]
  data.loc[i:i, 'ant_l_2']=sorted(count_int_L, reverse=True)[1:2]
  data.loc[i:i, 'ant_l_3']=sorted(count_int_L, reverse=True)[2:3]
  data.loc[i:i, 'ant_l_4']=sorted(count_int_L, reverse=True)[3:4]
  data.loc[i:i, 'ant_l_5']=sorted(count_int_L, reverse=True)[4:5]


  data.loc[i:i, 'post.ethmoid R gray/black ratio']=np.mean(count_post_R)
  data.loc[i:i, 'post_r_std']=np.std(count_post_R)
  data.loc[i:i, 'post_r_1']=sorted(count_post_R, reverse=True)[:1]
  data.loc[i:i, 'post_r_2']=sorted(count_post_R, reverse=True)[1:2]
  data.loc[i:i, 'post_r_3']=sorted(count_post_R, reverse=True)[2:3]
  data.loc[i:i, 'post_r_4']=sorted(count_post_R, reverse=True)[3:4]
  data.loc[i:i, 'post_r_5']=sorted(count_post_R, reverse=True)[4:5]

  data.loc[i:i, 'post.ethmoid L gray/black ratio']=np.mean(count_post_L)
  data.loc[i:i, 'post_l_std']=np.std(count_post_L)
  data.loc[i:i, 'post_l_1']=sorted(count_post_L, reverse=True)[:1]
  data.loc[i:i, 'post_l_2']=sorted(count_post_L, reverse=True)[1:2]
  data.loc[i:i, 'post_l_3']=sorted(count_post_L, reverse=True)[2:3]
  data.loc[i:i, 'post_l_4']=sorted(count_post_L, reverse=True)[3:4]
  data.loc[i:i, 'post_l_5']=sorted(count_post_L, reverse=True)[4:5]


  data.loc[i:i, 'frontal gray/black ratio']=np.mean(count_frontal)
  data.loc[i:i, 'front_std']=np.std(count_frontal)
  data.loc[i:i, 'front_1']=sorted(count_frontal, reverse=True)[:1]
  data.loc[i:i, 'front_2']=sorted(count_frontal, reverse=True)[1:2]
  data.loc[i:i, 'front_3']=sorted(count_frontal, reverse=True)[2:3]
  data.loc[i:i, 'front_4']=sorted(count_frontal, reverse=True)[3:4]
  data.loc[i:i, 'front_5']=sorted(count_frontal, reverse=True)[4:5]

  data.loc[i:i, 'sphenoid R gray/black ratio']=np.mean(count_sphenoid_R)
  data.loc[i:i, 'sphen_r_std']=np.std(count_sphenoid_R)
  data.loc[i:i, 'sphen_r_1']=sorted(count_sphenoid_R, reverse=True)[:1]
  data.loc[i:i, 'sphen_r_2']=sorted(count_sphenoid_R, reverse=True)[1:2]
  data.loc[i:i, 'sphen_r_3']=sorted(count_sphenoid_R, reverse=True)[2:3]
  data.loc[i:i, 'sphen_r_4']=sorted(count_sphenoid_R, reverse=True)[3:4]
  data.loc[i:i, 'sphen_r_5']=sorted(count_sphenoid_R, reverse=True)[4:5]

  data.loc[i:i, 'sphenoid L gray/black ratio']=np.mean(count_sphenoid_L)
  data.loc[i:i, 'sphen_l_std']=np.std(count_sphenoid_L)
  data.loc[i:i, 'sphen_l_1']=sorted(count_sphenoid_L, reverse=True)[:1]
  data.loc[i:i, 'sphen_l_2']=sorted(count_sphenoid_L, reverse=True)[1:2]
  data.loc[i:i, 'sphen_l_3']=sorted(count_sphenoid_L, reverse=True)[2:3]
  data.loc[i:i, 'sphen_l_4']=sorted(count_sphenoid_L, reverse=True)[3:4]
  data.loc[i:i, 'sphen_l_5']=sorted(count_sphenoid_L, reverse=True)[4:5]


  data.loc[i:i, 'septum area']=np.mean(count_septum)
  data.loc[i:i, 'sept_std']=np.std(count_septum)
  data.loc[i:i, 'sept_1']=sorted(count_septum, reverse=True)[:1]
  data.loc[i:i, 'sept_2']=sorted(count_septum, reverse=True)[1:2]
  data.loc[i:i, 'sept_3']=sorted(count_septum, reverse=True)[2:3]
  data.loc[i:i, 'sept_4']=sorted(count_septum, reverse=True)[3:4]
  data.loc[i:i, 'sept_5']=sorted(count_septum, reverse=True)[4:5]

  data.loc[i:i, 'inferior turbinate ratio']=np.mean(count_inferior_turbinate)
  data.loc[i:i, 'it_std']=np.std(count_inferior_turbinate)
  data.loc[i:i, 'it_1']=sorted(count_inferior_turbinate, reverse=True)[:1]
  data.loc[i:i, 'it_2']=sorted(count_inferior_turbinate, reverse=True)[1:2]
  data.loc[i:i, 'it_3']=sorted(count_inferior_turbinate, reverse=True)[2:3]
  data.loc[i:i, 'it_4']=sorted(count_inferior_turbinate, reverse=True)[3:4]
  data.loc[i:i, 'it_5']=sorted(count_inferior_turbinate, reverse=True)[4:5]


  data.loc[i:i, 'concha_blk_ratio']=np.mean(count_concha_black)
  data.loc[i:i, 'concha_blk_std']=np.std(count_concha_black)
  data.loc[i:i, 'concha_blk_1']=sorted(count_concha_black, reverse=True)[:1]
  data.loc[i:i, 'concha_blk_2']=sorted(count_concha_black, reverse=True)[1:2]
  data.loc[i:i, 'concha_blk_3']=sorted(count_concha_black, reverse=True)[2:3]
  data.loc[i:i, 'concha_blk_4']=sorted(count_concha_black, reverse=True)[3:4]
  data.loc[i:i, 'concha_blk_5']=sorted(count_concha_black, reverse=True)[4:5]



  data.loc[i:i, 'concha_size_ratio']=np.mean(count_concha_size)
  data.loc[i:i, 'concha_size_std']=np.std(count_concha_size)
  data.loc[i:i, 'concha_size_1']=sorted(count_concha_size, reverse=True)[:1]
  data.loc[i:i, 'concha_size_2']=sorted(count_concha_size, reverse=True)[1:2]
  data.loc[i:i, 'concha_size_3']=sorted(count_concha_size, reverse=True)[2:3]
  data.loc[i:i, 'concha_size_4']=sorted(count_concha_size, reverse=True)[3:4]
  data.loc[i:i, 'concha_size_5']=sorted(count_concha_size, reverse=True)[4:5]

  count_nasopharynx=count_nasopharynx[:10]
  data.loc[i:i, 'nasopharynx area']=np.mean(count_nasopharynx)
  data.loc[i:i, 'naso_std']=np.std(count_nasopharynx)
  data.loc[i:i, 'naso_1']=sorted(count_nasopharynx, reverse=True)[:1]
  data.loc[i:i, 'naso_2']=sorted(count_nasopharynx, reverse=True)[1:2]
  data.loc[i:i, 'naso_3']=sorted(count_nasopharynx, reverse=True)[2:3]
  data.loc[i:i, 'naso_4']=sorted(count_nasopharynx, reverse=True)[3:4]
  data.loc[i:i, 'naso_5']=sorted(count_nasopharynx, reverse=True)[4:5]


  data.to_csv("dataset_fill.csv")



In [24]:
data

,Case_ID,Hospital_ID,Age,Gender,maxillaryR score,maxillary R gray/black ratio,m_r_std,m_r_1,m_r_2,m_r_3,m_r_4,m_r_5,maxillary L score,maxillary L gray/black ratio,m_l_std,m_l_1,m_l_2,m_l_3,m_l_4,m_l_5,frontal score,frontal gray/black ratio,front_std,front_1,front_2,front_3,front_4,front_5,ant.ethmoid R score,ant.ethmoid R gray/black ratio,ant_r_std,ant_r_1,ant_r_2,ant_r_3,ant_r_4,ant_r_5,ant.ethmoid L score,ant.ethmoid L gray/black ratio,ant_l_std,ant_l_1,ant_l_2,ant_l_3,ant_l_4,ant_l_5,post.ethmoid R score,post.ethmoid R gray/black ratio,post_r_std,post_r_1,post_r_2,post_r_3,post_r_4,post_r_5,post.ethmoid L score,post.ethmoid L gray/black ratio,post_l_std,post_l_1,post_l_2,post_l_3,post_l_4,post_l_5,sphenoid R score,sphenoid R gray/black ratio,sphen_r_std,sphen_r_1,sphen_r_2,sphen_r_3,sphen_r_4,sphen_r_5,sphenoid L score,sphenoid L gray/black ratio,sphen_l_std,sphen_l_1,sphen_l_2,sphen_l_3,sphen_l_4,sphen_l_5,Osteomeatal complex R score,Osteomeatal complex L score,septum score,septum area,sept_std,sept_1,sept_2,sept_3,sept_4,sept_5,inferior turbinate score,inferior turbinate ratio,it_std,it_1,it_2,it_3,it_4,it_5,concha bullosa score,concha_blk_ratio,concha_blk_std,concha_blk_1,concha_blk_2,concha_blk_3,concha_blk_4,concha_blk_5,concha_size_ratio,concha_size_std,concha_size_1,concha_size_2,concha_size_3,concha_size_4,concha_size_5,nasopharynx score,nasopharynx area,naso_std,naso_1,naso_2,naso_3,naso_4,naso_5
0,1,KA00220503,70.0,M,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,KA00220955,49.0,M,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,KA00222158,45.0,F,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,KA00222724,57.0,M,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,KA00223460,24.0,F,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

#models

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
with open('models 70.pkl', 'rb') as f:
    trained_models = pickle.load(f)

#lists 

##data

In [ ]:
septum_data_col=['septum score','septum area','sept_std','sept_1','sept_2','sept_3','sept_4','sept_5',
               'inferior turbinate score','inferior turbinate ratio','it_std','concha bullosa score','concha_size_ratio','concha_size_std']

inferior_data_col=['inferior turbinate score','inferior turbinate ratio','it_std','it_1','it_2','it_3','it_4','it_5']

nasopharynx_data_col=['nasopharynx score','nasopharynx area','naso_std','naso_1','naso_2','naso_3','naso_4','naso_5']

concha_data_col=['concha bullosa score','concha_blk_ratio','concha_blk_std','concha_blk_1','concha_blk_2','concha_blk_3','concha_blk_4','concha_blk_5',
                 'concha_size_ratio','concha_size_std','concha_size_1','concha_size_2','concha_size_3','concha_size_4','concha_size_5']

Osteomeatal_r_data_col=['Osteomeatal complex R score','maxillary R gray/black ratio','m_r_std','ant.ethmoid R gray/black ratio',
                        'ant_r_std','maxillaryR score','ant.ethmoid R score']

Osteomeatal_l_data_col=['Osteomeatal complex L score','maxillary L gray/black ratio','m_l_std','ant.ethmoid L gray/black ratio',
                        'ant_l_std','maxillary L score','ant.ethmoid L score']

frontal_data_col=['frontal score','frontal gray/black ratio','front_std','front_1','front_2','front_3','front_4','front_5']

sphenoid_r_data_col=['sphenoid R score','sphenoid R gray/black ratio','sphen_r_std','sphen_r_1','sphen_r_2','sphen_r_3','sphen_r_4','sphen_r_5']

sphenoid_l_data_col=['sphenoid L score','sphenoid L gray/black ratio','sphen_l_std','sphen_l_1','sphen_l_2','sphen_l_3','sphen_l_4','sphen_l_5']

post_r_data_col=['post.ethmoid R score','post.ethmoid R gray/black ratio','post_r_std','post_r_1','post_r_2','post_r_3','post_r_4','post_r_5']

post_l_data_col=['post.ethmoid L score','post.ethmoid L gray/black ratio','post_l_std','post_l_1','post_l_2','post_l_3','post_l_4','post_l_5']

ant_r_data_col=['ant.ethmoid R score','ant.ethmoid R gray/black ratio','ant_r_std','ant_r_1','ant_r_2','ant_r_3','ant_r_4','ant_r_5']

ant_l_data_col=['ant.ethmoid L score','ant.ethmoid L gray/black ratio','ant_l_std','ant_l_1','ant_l_2','ant_l_3','ant_l_4','ant_l_5']

maxillary_r_data_col=['maxillaryR score','maxillary R gray/black ratio','m_r_std','m_r_1','m_r_2','m_r_3','m_r_4','m_r_5']

maxillary_l_data_col=['maxillary L score','maxillary L gray/black ratio','m_l_std','m_l_1','m_l_2','m_l_3','m_l_4','m_l_5']




##prediction

In [ ]:
septum_df_col=['septum_score','septum_area','septum_std','septum_1','septum_2','septum_3','septum_4','septum_5',
               'inferior_score','inferior_ratio','inferior_std','concha_score','concha_size_ratio','concha_size_std']

inferior_df_col=['inferior_score','inferior_ratio','inferior_std','inferior_1','inferior_2','inferior_3','inferior_4','inferior_5']

nasopharynx_df_col=['nasopharynx_score','nasopharynx_area','naso_std','naso_1','naso_2','naso_3','naso_4','naso_5']

concha_df_col=['concha_score','concha_blk_ratio','concha_blk_std','concha_blk_1','concha_blk_2','concha_blk_3','concha_blk_4','concha_blk_5',
                    'concha_size_ratio','concha_size_std','concha_size_1','concha_size_2','concha_size_3','concha_size_4','concha_size_5']

Osteomeatal_df_col=['osteomeatal_score','maxillary_ratio','maxillary_std','ant_ethmoid_ratio','ant_ethmoid_std','maxillary_score','ant_ethmoid_score']

frontal_df_col=['frontal_score','frontal_ratio','frontal_std','frontal_1','frontal_2','frontal_3','frontal_4','frontal_5']

sphenoid_df_col=['sphenoid_score','sphenoid_ratio','sphenoid_std','sphenoid_1','sphenoid_2','sphenoid_3','sphenoid_4','sphenoid_5']

post_df_col=['post_ethmoid_score','post_ethmoid_ratio','post_ethmoid_std','post_ethmoid_1','post_ethmoid_2','post_ethmoid_3','post_ethmoid_4','post_ethmoid_5']

ant_df_col=['ant_ethmoid_score','ant_ethmoid_ratio','ant_ethmoid_std','ant_ethmoid_1','ant_ethmoid_2','ant_ethmoid_3','ant_ethmoid_4','ant_ethmoid_5']

maxillary_df_col=['maxillary_score','maxillary_ratio','maxillary_std','maxillary_1','maxillary_2','maxillary_3','maxillary_4','maxillary_5']

#predictions

##maxillary_prediction

In [ ]:
df_maxillary = pd.DataFrame()

for (i, n) in zip(maxillary_df_col[1:], maxillary_r_data_col[1:]):
  df_maxillary[i]= data[n]

df_maxillary.reset_index(inplace=True)

m_r_p=trained_models['maxillary'].predict(df_maxillary)

#-----------------------------------------------------------------------------------

df_maxillary = pd.DataFrame()

for (i, n) in zip(maxillary_df_col[1:], maxillary_l_data_col[1:]):
  df_maxillary[i]= data[n]

df_maxillary.reset_index(inplace=True)

m_l_p=trained_models['maxillary'].predict(df_maxillary)

##frontal_prediction

In [ ]:
df_frontal = pd.DataFrame()

for (i, n) in zip(frontal_df_col[1:], frontal_data_col[1:]):
  df_frontal[i]= data[n]

df_frontal.reset_index(inplace=True)

f_p=m_l_p=trained_models['frontal'].predict(df_frontal)


##ant_prediction

In [ ]:
df_ant_ethmoid = pd.DataFrame()

for (i, n) in zip(ant_df_col[1:], ant_r_data_col[1:]):
  df_ant_ethmoid[i]= data[n]

df_ant_ethmoid.reset_index(inplace=True)

ant_r_p=trained_models['ant_ethmoid'].predict(df_ant_ethmoid)

#------------------------------------------------------------------------------------------

df_ant_ethmoid = pd.DataFrame()

for (i, n) in zip(ant_df_col[1:], ant_l_data_col[1:]):
  df_ant_ethmoid[i]= data[n]

df_ant_ethmoid.reset_index(inplace=True)  

ant_l_p=trained_models['ant_ethmoid'].predict(df_ant_ethmoid)

##post_prediction

In [ ]:
df_post_ethmoid = pd.DataFrame()

for (i, n) in zip(post_df_col[1:], post_r_data_col[1:]):
  df_post_ethmoid[i]= data[n]

df_post_ethmoid.reset_index(inplace=True)  

post_r_p=trained_models['post_ethmoid'].predict(df_post_ethmoid)

#------------------------------------------------------------------------------------------

df_post_ethmoid = pd.DataFrame()

for (i, n) in zip(post_df_col[1:], post_l_data_col[1:]):
  df_post_ethmoid[i]= data[n]

df_post_ethmoid.reset_index(inplace=True)  

post_l_p=trained_models['post_ethmoid'].predict(df_post_ethmoid)

##sphenoid_prediction

In [ ]:
df_sphenoid = pd.DataFrame()

for (i, n) in zip(sphenoid_df_col[1:], sphenoid_r_data_col[1:]):
  df_sphenoid[i]= data[n]

df_sphenoid.reset_index(inplace=True)

sphen_r_p=trained_models['sphenoid'].predict(df_sphenoid)

#-----------------------------------------------------------------------------

df_sphenoid = pd.DataFrame()

for (i, n) in zip(sphenoid_df_col[1:], sphenoid_l_data_col[1:]):
  df_sphenoid[i]= data[n]

df_sphenoid.reset_index(inplace=True)

sphen_l_p=trained_models['sphenoid'].predict(df_sphenoid)

##Osteomeatal_prediction

In [ ]:
df_Osteomeatal = pd.DataFrame()

for (i, n) in zip(Osteomeatal_df_col[1:], Osteomeatal_r_data_col[1:]):
  df_Osteomeatal[i]= data[n]

df_Osteomeatal['ant_ethmoid_score']=ant_r_p
df_Osteomeatal['maxillary_score']=m_r_p

df_Osteomeatal.reset_index(inplace=True)

ost_r_p=trained_models['osteomeatal'].predict(df_Osteomeatal)

#------------------------------------------------------------------

df_Osteomeatal = pd.DataFrame()

for (i, n) in zip(Osteomeatal_df_col[1:], Osteomeatal_l_data_col[1:]):
  df_Osteomeatal[i]= data[n]

df_Osteomeatal['ant_ethmoid_score']=ant_l_p
df_Osteomeatal['maxillary_score']=m_l_p

df_Osteomeatal.reset_index(inplace=True)

ost_l_p=trained_models['osteomeatal'].predict(df_Osteomeatal)


##concha_prediction

In [ ]:
df_concha = pd.DataFrame()

for (i, n) in zip(concha_df_col[1:], concha_data_col[1:]):
  df_concha[i]= data[n]

df_concha.reset_index(inplace=True)


concha_p=trained_models['concha'].predict(df_concha)

##nasopharynx_prediction

In [ ]:
df_nasopharynx = pd.DataFrame()

for (i, n) in zip(nasopharynx_df_col[1:], nasopharynx_data_col[1:]):
  df_nasopharynx[i]= data[n]

df_nasopharynx.reset_index(inplace=True)

naso_p=trained_models['nasopharynx'].predict(df_nasopharynx)

##inferior_prediction

In [ ]:
df_inferior = pd.DataFrame()

for (i, n) in zip(inferior_df_col[1:], inferior_data_col[1:]):
  df_inferior[i]= data[n]

df_inferior.reset_index(inplace=True)

infer_p=trained_models['inferior'].predict(df_inferior)

##septum_prediction

In [ ]:
df_septum = pd.DataFrame()

for (i, n) in zip(septum_df_col[1:], septum_data_col[1:]):
  df_septum[i]= data[n]

df_septum['inferior_score']= infer_p
df_septum['concha_score']= concha_p


df_septum.reset_index(inplace=True)


sept_p=trained_models['septum'].predict(df_septum)

##final_prediction_dataset

In [ ]:
prediction = pd.DataFrame()
sinuses_score_p=m_r_p+m_l_p+ant_r_p+ant_l_p+post_r_p+post_l_p+sphen_r_p+sphen_l_p+ost_r_p+ost_l_p
prediction['sinuses_score_p'] = sinuses_score_p
prediction['sinuses_score_p'] = prediction['sinuses_score_p'].apply(lambda x: 0 if x < 18 else 2)
prediction['sergury_p'] = 0
prediction['sept_p']=sept_p
prediction['infer_p']=infer_p
prediction['concha_p']=concha_p
prediction['naso_p']=naso_p
prediction.loc[prediction['sinuses_score_p'] ==18, 'sergury_p'] = 2
prediction.loc[prediction['naso_p'] ==2, 'sergury_p'] = 2
prediction.loc[prediction['infer_p'] ==2, 'sergury_p'] = 2
prediction.loc[prediction['concha_p'] ==2, 'sergury_p'] = 2
prediction.loc[prediction['naso_p'] ==2, 'sergury_p'] = 2

##calculate

In [ ]:
data.head(50)

In [ ]:
data.to_csv("dataset_fill.csv")